In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms

transform = transforms.ToTensor()

In [3]:
mbti_df = pd.read_csv('crop128/all.csv')

photo_df = pd.DataFrame(columns=['photo'], dtype= 'object')

# put the photo data into a dataframe
for i in range(len(mbti_df)):
    image = transform(Image.open('crop128/' + mbti_df['filename'][i]))
    photo_df.loc[i] = [image]

mbti_df = mbti_df['MBTI']

mbti_df.head()

0    enfj
1    enfj
2    enfj
3    enfj
4    enfj
Name: MBTI, dtype: object

In [4]:
photo_df.iloc[0][0].shape

torch.Size([1, 64, 64])

In [5]:
alphabet = ['t', 'f']

# if mbti_df['MBTI'] includes alphabet[0], then mbti_df['MBTI'] = 0, else 1
mbti_df = mbti_df.apply(lambda x: 1 if alphabet[0] in x else 0)

mbti_df.head()

0    0
1    0
2    0
3    0
4    0
Name: MBTI, dtype: int64

In [6]:
#print the statistics of the mbti_df
print(mbti_df.value_counts())

1    1551
0    1493
Name: MBTI, dtype: int64


In [7]:
# train_data includes the element of 2-dim tensor
train_data = photo_df['photo'].values
train_label = mbti_df.values

In [8]:
#전체 data 중 train의 비율
train_ratio = 0.8

train_idx = np.random.choice(len(train_data), int(len(train_data) * train_ratio), replace=False)
test_idx = np.array(list(set(range(len(train_data))) - set(train_idx)))

test_data = train_data[test_idx]
test_label = train_label[test_idx]

train_data = train_data[train_idx]
train_label = train_label[train_idx]

In [21]:
from bayes_opt import BayesianOptimization
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

class MBTI_Dataset(Dataset):
    def __init__(self, train_label, train_data):
        self.train_label = train_label
        self.train_data = train_data

    def __len__(self):
        return len(self.train_label)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        mbti = self.train_label[idx]
        photo = self.train_data[idx]

        return mbti, photo

# CNN model
class CNN(nn.Module):
    def __init__(self):
        # 5 layer convolutional neural network
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 2, 1)
        self.conv2 = nn.Conv2d(32, 64, 2, 1)
        self.conv3 = nn.Conv2d(64, 64, 2, 1)
        self.conv4 = nn.Conv2d(64, 64, 2, 1)
        self.conv5 = nn.Conv2d(64, 32, 2, 1)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(1024 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 2)
        self.pool = nn.MaxPool2d(2, 2)


    def forward(self, x):
        # 7 layer convolutional neural network
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = self.fc5(x)
        return x


# find best parameters using bayesian optimization
def bayes_opt(train_data, train_label, test_data, test_label):
    def train_CNN(lr, batch_size, epoch):
        # train data
        train_dataset = MBTI_Dataset(train_label, train_data)
        train_loader = DataLoader(train_dataset, batch_size=int(batch_size), shuffle=True)

        # test data
        test_dataset = MBTI_Dataset(test_label, test_data)
        test_loader = DataLoader(test_dataset, batch_size=int(batch_size), shuffle=True)

        # CNN model
        model = CNN()

        # loss function
        criterion = nn.CrossEntropyLoss()

        # optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        # train
        for e in range(int(epoch)):
            running_loss = 0
            for mbti, photo in train_loader:
                optimizer.zero_grad()
                output = model(photo)
                loss = criterion(output, mbti)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
            else:
                print("Epoch {} - Training loss: {}".format(e, running_loss / len(train_loader)))

        # test
        correct = 0
        total = 0
        with torch.no_grad():
            for mbti, photo in test_loader:
                output = model(photo)
                _, predicted = torch.max(output.data, 1)
                total += mbti.size(0)
                correct += (predicted == mbti).sum().item()

        print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

        return correct / total

    optimizer = BayesianOptimization(
        f=train_CNN,
        pbounds={"lr": (0.0001, 0.1), "batch_size": (1, 100), "epoch": (1, 100)},
        random_state=1,
    )

    optimizer.maximize(init_points=2, n_iter=10)

    print(optimizer.max)

bayes_opt(train_data, train_label, test_data, test_label)


|   iter    |  target   | batch_... |   epoch   |    lr     |
-------------------------------------------------------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (42x32 and 16384x512)

In [ ]:
# print roc auc score
def roc_auc_score(test_label, test_data):
    # train data
    train_dataset = MBTI_Dataset(train_label, train_data)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

    # test data
    test_dataset = MBTI_Dataset(test_label, test_data)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    # CNN model
    model = CNN()

    # loss function
    criterion = nn.CrossEntropyLoss()

    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    # train
    for e in range(1):
        running_loss = 0
        for mbti, photo in train_loader:
            optimizer.zero_grad()
            output = model(photo)
            loss = criterion(output, mbti)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        else:
            print("Epoch {} - Training loss: {}".format(e, running_loss / len(train_loader)))

    # test
    correct = 0
    total = 0
    with torch.no_grad():
        for mbti, photo in test_loader:
            output = model(photo)
            _, predicted = torch.max(output.data, 1)
            total += mbti.size(0)
            correct += (predicted == mbti).sum().item()

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

    # roc auc score
    y_pred = model(test_data)
    y_pred = y_pred.detach().numpy()
    y_pred = np.argmax(y_pred, axis=1)
    y_test = test_label.detach().numpy()
    print("roc auc score: ", roc_auc_score(y_test, y_pred))

roc_auc_score(test_label, test_data)